<a href="https://colab.research.google.com/github/ParthPanchall/Pneumonia-Detection-Deployment/blob/master/Pneumonia_Detection_Using_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Step 1`: Importing the dataset

In [1]:
# Installing kaggle
!pip install -q kaggle

In [2]:
# To upload kaggle json file from the local systems
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chandurobbi","key":"29e7a02a2948be67ac9c3b5c3f5e9937"}'}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Create a kaggle directory
!mkdir ~/.kaggle

In [5]:


# Copy the kaggle.json to folder created
!cp kaggle.json ~/.kaggle/

In [6]:
# Permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 52% 1.19G/2.29G [00:05<00:05, 234MB/s]

# `Step 2`: Unzip the dataset and make directories

In [ ]:
!unzip chest-xray-pneumonia.zip

In [ ]:
# Import the helper function and  get the functions needed that we are going to need in this notebook
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
from helper_functions import walk_through_dir, plot_loss_curves

In [ ]:
import shutil
shutil.rmtree("/content/chest_xray/__MACOSX")
shutil.rmtree("/content/chest_xray/chest_xray")

In [ ]:
walk_through_dir("/content/chest_xray")

In [ ]:
# Visualize the images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os

def view_random_images(target_dir, target_class):
  # Setup the target directory
  target_folder = target_dir + target_class

  # Get the random image path
  rand_img = random.sample(os.listdir(target_folder), 1)

  # Read the image and plot
  img = mpimg.imread(target_folder + "/" + rand_img[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis(False);

  print(f"Image shape :{img.shape}")

  return img

In [ ]:
img = view_random_images(target_dir="/content/chest_xray/train/",
                         target_class="PNEUMONIA")

In [ ]:
img = view_random_images(target_dir="/content/chest_xray/train/",
                         target_class="NORMAL")

# Preparing the data for modelling


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# 1. Preprocessing the data (getting the values between 0 and 1)
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)
val_datagen = ImageDataGenerator(rescale=1/255.)

# 2. Setting and getting the path of our directories
train_dir = "/content/chest_xray/train"
test_dir = "/content/chest_xray/test"
val_dir = "/content/chest_xray/val"

# 3. Importing the data from the directories and turn it into the batches
print("Train Data")
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                                                 batch_size=32,
                                                                 target_size=(224, 224),
                                                                 class_mode="binary",
                                                                 seed=42)

print("Test Data")
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                                                batch_size=32,
                                                                target_size=(224,224),
                                                                class_mode="binary",
                                                                seed=42)

print("Val Data")
val_data = val_datagen.flow_from_directory(directory=val_dir,
                                                                batch_size=32,
                                                                target_size=(224,224),
                                                                class_mode="binary",
                                                                seed=42)



In [ ]:
!pip install gradio

# `Step 3`: Modelling


In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/tuned_resnet.h5")

In [ ]:
model.summary()

In [ ]:
model.evaluate(test_data)

In [ ]:
import numpy as np
def pneumoniaPrediction(img):
    img = np.array(img)/255
    img = img.reshape(1, 224, 224, 3)
    isPneumonic = model.predict(img)[0]
    imgClass = "Normal" if isPneumonic<0.5 else "Pneumonic"
    return imgClass

In [ ]:
pr = model.predict(test_data)
for i in range(len(pr)):
    if pr[i]>0.5:
        pr[i]=1
    else:
        pr[i]=0

In [ ]:
import gradio as gr
img = gr.inputs.Image(shape=(224,224))
label = gr.outputs.Label(num_top_classes=1)

In [ ]:
interface = gr.Interface(fn = pneumoniaPrediction,
                            title = "Pneumonia Detection using Chest X-Ray",
                            inputs = img,
                            outputs = label,
                            interpretation = "default")
interface.launch(debug=True, share=True)